# Лаб 1. Корреляционная система

При создании задания использовались Visual Studio Code и Python 3.8.

## Теория

Данные для рекомендательных систем представляют из себя матрицу $R$ юзер-айтемов, в строках которой находятся пользователи, а в столбцах объекты. В ячейках матрицы находятся оценки пользователей (или какая-то другая метрика $-$ просмотры, покупки и т.д.):

$$
R = \begin{pmatrix}
   r_{1 1} & r_{1 2} & ...     &    \\
   r_{2 1} & r_{2 1} & ...     &     \\
   ...     & ...     & r_{u i} & ... \\
           &         & ...     &     \\
\end{pmatrix}
$$

Матрица $R$ сильно разрежена, то есть большая часть ячеек пуста, поскольку каждый пользователь взаимодействовал с очень малым числом объектов. Задача $-$ предсказать значения в пустых ячейках, то есть получить новую заполненную матрицу $\hat R$, максимально похожую на $R$.

### Обозначения

- $U - $ множество пользователей;
- $I - $ множество айтемов (объектов);
- $I(u)$, где $u \in U -$ множество айтемов, которое оценил пользователь $u$. То есть такие айтемы, для которых в строке $u$ матрицы $R$ не пусто. Аналогично $I(u, v) -$ множество объектов, которые оценили и $u$ и $v$;
- $U(u)$, где $u \in U -$ Множество пользователей, оценивали то же что и $u$, то есть $I(u) \cap I(v) \ne \empty$, где $v \in U(u)$;
- $r_u$, где $u \in U -$ строка матрицы $R$, соответствующая пользователю $u$;
- $r_i$, где $i \in I -$ столбец матрицы $R$, соответствующий айтему $i$;
- $\bar r_u$, где $u \in U -$ среднее значение по всем заполненным оценкам пользователя $u$.


### Корреляционная модель

Один из самых простых способов $-$ использовать взвешенное среднее. То есть, чтобы определить оценку пользователя $u$ айтема $i$, надо усреднить оценки всех пользователей, посмотревших этот фильм (user-based подход). При этом будем учитывать оценку пользователя с большим коэффициентом если пользователь "похож" на нашего:

$$
\hat{r}_{ui} = \frac{
        \sum_{v \in U(u)} S(u, v)\cdot r_v
    } {
        \sum_{v \in U(u)} S(u, v)
    }
$$

Здесь $S$ это функция близости, которая тем больше, чем более "похожи" пользователи друг на друга.

Поскольку оценки разных пользователей могут отличаться $-$ кто-то ставит всем фильмам 9 и 10, а кто-то 0 и 1, можно попытаться устранить проблему, предсказывая не само значение $r_u$, а отклонение от среднего значения $(r_u - \bar{r}_u)$:

$$
\hat{r}_{ui} = \bar{r}_{u} + \frac{
        \sum_{v \in U(u)} S(u, v) \cdot (r_v - \bar r_v)
    } {
        \sum_{v \in U(u)} S(u, v)
    }
$$



### Функция сходства

Будем рассматривать функцию сходства двух юзером для user-based модели, функции сходства для айтемов определяется аналогично.

#### Косинусная мера сходства

Считаем косинус угла между пользователями в пространстве определяющих их векторов. То есть берём скалярное произведение и делим на длины векторов. Единственное, на что следует обратит внимание, что берутся не все оценки пользователей, а только для тех айтемов, которые они оба оценили:

$$
S(u, v) = \frac{
        \sum_{i \in I(u, v)} r_{ui}r_{vi}
    }{
        \sqrt{\sum_{i \in I(u)} r_{ui}^2}
        \sqrt{\sum_{i \in I(v)} r_{vi}^2}
    }
$$


## Задание

Дан датасет с оценками пользователями фильмов. Реализуйте алгоритм в соответствии с вашим вариантом и порекомендуйте себе фильмы. Сделайте выводы.

В датасете фильмы оценены по пятибальной шкале. Если в вашем варианте используется функция сходства для бинарных данных, используйте факт просмотра фильма (наличие оценки вообще).

### Варианты

1. User-based подход, Косинусная мера сходства.
2. User-based подход, Корреляция Пирсона.
3. User-based подход, Мера близости Жаккара.
4. Item-based подход, Косинусная мера сходства.
5. Item-based подход, Корреляция Пирсона
6. Item-based подход, Мера близости Жаккара.


## Код

Устанавливаем библиотеки

In [1]:
#%pip install pandas tqdm

Импорты

In [18]:
# Пандас нам нужен для загрузки csv и работы с матрицей
import pandas as pd
# Это библиотека для визуализации прогресса в питоновском ноутбуке
from tqdm.notebook import tqdm

import math
import time
import numpy as np

Загружаем датасет

In [19]:
# Сам датасет
ratings = pd.read_csv('./ml-latest-small/ratings.csv', delimiter=',')
# Оставляем в таблице только нужные столбцы
ratings = ratings.loc[:, ["userId", "movieId", "rating"]]

# Строка в таблице в датасета это id пользователя, id фильма, рейтинг
# На id пользователей нам плевать, а фильмы хочется смотреть по названиям,
# поэтому загружаем табличку сопоставления названий фильмов их id
movies = pd.read_csv('./ml-latest-small/movies.csv', delimiter=',')

In [20]:
# Делаем таблицу для преобразования имени в id
title_to_id = movies.loc[:, ["title", "movieId"]]
title_to_id.set_index("title", inplace=True)

# Делаем таблицу для преобразования id в имя
id_to_title = movies.loc[:, ["movieId", "title"]]
id_to_title.set_index("movieId", inplace=True)

# Проверяем что всё работает
print(title_to_id.loc[["Iron Man (2008)", "Doctor Who: A Christmas Carol (2010)"], :])
print(id_to_title.loc[[59315, 147376], :])


                                      movieId
title                                        
Iron Man (2008)                         59315
Doctor Who: A Christmas Carol (2010)   147376
                                        title
movieId                                      
59315                         Iron Man (2008)
147376   Doctor Who: A Christmas Carol (2010)


In [21]:
# Мы хотим получить для себя какую-то рекомендацию,
# для этого оценим несколько фильмов по пятибальной шкале
# (аккуратно копипастим имена из датасета)
my_ratings = {
        "Toy Story (1995)": 3.0,
        "Jumanji (1995)": 2.0,
        "It Takes Two (1995)": 1.0,
        "Mortal Kombat (1995)": 2.0,
        "Taxi Driver (1976)": 5.0,
        "Scarface (1983)": 5.0,
        "Saw (2004)": 4.0,
      "xXx: State of the Union (2005)": 0.0,
      "V for Vendetta (2006)": 4.0,
      "Night at the Museum (2006)": 3.0,
      "Pirates of the Caribbean: At World's End (2007)": 5.0,
    "Harry Potter and the Order of the Phoenix (2007)":1.0,
    "Resident Evil: Extinction (2007)":3.0,
    "Zebraman (2004)":1.0,
    "Max Payne (2008)":3.0,
    "Madagascar: Escape 2 Africa (2008)":4.0,
    "Shrek the Halls (2007)":2.0,
    "Drive (2011)":5.0,
    "Casino (1995)":5.0,
    "LEGO Batman: The Movie - DC Heroes Unite (2013)": 4.0,
    "Lord of the Rings: The Fellowship of the Ring, The (2001)": 4.0,
    "Back to the Future (1985)": 3.0,
    "Iron Man (2008)": 4.0,
}

# Даём нашему юзеру id, которого нет в датасете
my_user_id = 666
# Докидываем свои оценки в датасет
for m, r in my_ratings.items():
    mid = title_to_id.loc[m]["movieId"]
    row = pd.DataFrame([[my_user_id, mid, r]], columns=["userId", "movieId", "rating"])
    ratings = pd.concat([ratings, row])
ratings=ratings.reset_index(drop=True)

In [22]:
# Тут реализуем функцию предсказания.
# Или класс. Главное - предсказать.

def r_mid(u):
    return ratings.loc[ratings['userId'] == u]["rating"].mean()
def predict(user_id, movie_id,s):
    pred=0      
    if (ratings.loc[ratings['userId'] == user_id]["movieId"].isin([movie_id]).any()):
        pred=ratings.loc[(ratings['userId'] == user_id)&(ratings['movieId']==movie_id)]['rating'].reset_index(drop=True)[0]
    else:
        r_up=0
        r_down=0
        
        for i in (ratings.loc[ratings['movieId'] == movie_id]['userId']):
            a= list(set( ratings.loc[ratings['userId'] == user_id]['movieId']).intersection(ratings.loc[ratings['userId'] == i]['movieId']))
            if a:
                #s=S(user_id,i,s_down1,s_down2)
                r_up += s[user_id]*(ratings.loc[(ratings['userId'] == i)&(ratings['movieId']==movie_id)]['rating'].reset_index(drop=True)[0]-r_mid(i))
                r_down+=abs(s[user_id])
        if r_down==0:
            pred=0.0
        else:
            pred=r_mid(user_id)+r_up/r_down
    #print (r_up)
    return pred


In [24]:
ratings.loc[ratings['movieId'] == 49]

,userId,movieId,rating
29386,202,49,3.0


In [23]:
from collections import Counter

def S (u,v,s_down1,s_down2):
    s_up=0
    a= list(set( ratings.loc[ratings['userId'] == u]['movieId']).intersection(ratings.loc[ratings['userId'] == v]['movieId']))
    for i in a:
        s_up+=(ratings.loc[(ratings['userId'] == v)&(ratings['movieId']==i)]['rating'].reset_index(drop=True)[0]*
               ratings.loc[(ratings['userId'] == u)&(ratings['movieId']==i)]['rating'].reset_index(drop=True)[0])
    return (s_up/((s_down1*s_down2[v])**(0.5)))

result = {}
s_down2={}

count=Counter(ratings['movieId'])
l= ratings.loc[ratings['userId'] == my_user_id]['rating']
s_down1=sum(i*i for i in l)
for i in ratings['userId'].drop_duplicates().reset_index(drop=True):
    q= ratings.loc[ratings['userId'] == i]['rating']
    s_down2[i]=sum(w*w for w in q)
s_all={}
for i in ratings['userId'].drop_duplicates().reset_index(drop=True):
    s_all[i]=S(my_user_id,i,s_down1,s_down2)
dict(sorted(s_all.items(), key=lambda item: item[1], reverse=True))

{666: 1.0,
 88: 0.15788386532904786,
 332: 0.1403062173797883,
 339: 0.1365045133615836,
 106: 0.1359503845466025,
 18: 0.13329801030663233,
 418: 0.12882961602640605,
 573: 0.12849128251758804,
 112: 0.12693365203291793,
 145: 0.12683240830711334,
 434: 0.12412940628698402,
 366: 0.1229365490954697,
 413: 0.12169267064663972,
 407: 0.12117999441166728,
 317: 0.12030714428894142,
 239: 0.119450092303686,
 103: 0.11832542235213546,
 362: 0.11682813087341475,
 581: 0.11146524440853545,
 320: 0.11093605699652605,
 291: 0.11089463755358162,
 249: 0.11046099625013522,
 585: 0.10913019119670214,
 107: 0.10791995308121104,
 393: 0.10777233134340677,
 399: 0.10776766197186351,
 373: 0.10681418668638457,
 380: 0.10669834023034785,
 401: 0.10611894655783179,
 444: 0.10578313170510119,
 137: 0.10565052450659362,
 298: 0.10522012063991765,
 219: 0.10505928774613704,
 62: 0.1049749514305962,
 551: 0.10327531836340792,
 307: 0.10241090852263145,
 466: 0.10218248033728093,
 378: 0.10110377114902176,


Даём предсказание для каждого фильма

In [15]:
for m in tqdm(ratings['movieId'].drop_duplicates().reset_index(drop=True)):
    if ratings.loc[ratings['userId'] == my_user_id]["movieId"].isin([m]).any() or count[m]<10:
        result[m]=-1
    else:
        result[m] = predict(my_user_id, m,s_all)
    #print(result.items())
result = {k: v for k, v in result.items()}

  0%|          | 0/9724 [00:00<?, ?it/s]

In [16]:
r_mid(my_user_id)

3.1739130434782608

Выводим результат

In [17]:
# Преобразуем id фильмов в нормальные названия
human_readable_result = {}
for m, v in result.items():
    title = id_to_title.loc[m]["title"]
    human_readable_result[title] = v

# Сортируем массив с результатами по убыванию
sorted_result = {k: v for k, v in sorted(human_readable_result.items(), key=lambda item: item[1], reverse=True) if v != 0.0}

# Выводим первые 20 рекомендаций
for m, v in list(sorted_result.items())[:30]:
    print(m, ": ", v)


Guess Who's Coming to Dinner (1967) :  4.164711325493841
Paths of Glory (1957) :  4.1073221987771005
Yojimbo (1961) :  4.053682649249413
Hamlet (1996) :  4.025431408013523
Touch of Evil (1958) :  4.019393977344609
Top Secret! (1984) :  4.019249905405833
Wallace & Gromit: The Best of Aardman Animation (1996) :  4.007942183702457
Lawrence of Arabia (1962) :  4.005500917752188
Streetcar Named Desire, A (1951) :  4.0046873692134834
Ran (1985) :  3.9964591485568173
His Girl Friday (1940) :  3.9600504814331363
Secrets & Lies (1996) :  3.9588223034784065
Kelly's Heroes (1970) :  3.9557239303602048
Drunken Master (Jui kuen) (1978) :  3.9544982322960687
Mary and Max (2009) :  3.9293980088906624
Shawshank Redemption, The (1994) :  3.914632474142481
Creature Comforts (1989) :  3.908872663804484
Grave of the Fireflies (Hotaru no haka) (1988) :  3.9081460277585496
High Noon (1952) :  3.9002813623808295
Double Indemnity (1944) :  3.8989738011242174
Ghost in the Shell (Kôkaku kidôtai) (1995) :  3.896

## Выводы

Объясните полученный результат:

Субъективно, выбранная рекомендательная имеет достаточно посредственный характер, но после того, что фильм должен быть оценен хотя бы 10 пользователями, выборка стала собираться более внятно.

In [ ]:
stop

первый вариант

In [ ]:
# Тут реализуем функцию предсказания.
# Или класс. Главное - предсказать.
def S (u,v):
    s_up=0
    s_down1=0
    s_down2=0
    a= list(set( ratings.loc[ratings['userId'] == u]['movieId']).intersection(ratings.loc[ratings['userId'] == v]['movieId']))
    for i in a:
        s_up+=(ratings.loc[(ratings['userId'] == v)&(ratings['movieId']==i)]['rating'].reset_index(drop=True)[0]*
               ratings.loc[(ratings['userId'] == u)&(ratings['movieId']==i)]['rating'].reset_index(drop=True)[0])
        s_down1+= ratings.loc[(ratings['userId'] == u)&(ratings['movieId']==i)]['rating'].reset_index(drop=True)[0]**2
        s_down2+= ratings.loc[(ratings['userId'] == v)&(ratings['movieId']==i)]['rating'].reset_index(drop=True)[0]**2
        #print (s_up/((s_down1*s_down1)**(0.5)))
    return (s_up/((s_down1*s_down2)**(0.5)))
def r_mid(u):
    return ratings.loc[ratings['userId'] == u]["rating"].mean()
def predict(user_id, movie_id):
    pred=0
    if (ratings.loc[ratings['userId'] == user_id]["movieId"].isin([movie_id]).any()):
        pred=ratings.loc[(ratings['userId'] == user_id)&(ratings['movieId']==movie_id)]['rating'].reset_index(drop=True)[0]
    else:
        r_up=0
        r_down=0
        for i in (ratings.loc[ratings['movieId'] == movie_id]['userId']):
            a= list(set( ratings.loc[ratings['userId'] == user_id]['movieId']).intersection(ratings.loc[ratings['userId'] == i]['movieId']))
            if a:
                s=S(user_id,i)
                r_up += s*(ratings.loc[(ratings['userId'] == i)&(ratings['movieId']==movie_id)]['rating'].reset_index(drop=True)[0]-r_mid(i))
                r_down+=abs(s)
        if r_down==0:
            pred=0.0
        else:
            pred=r_mid(user_id)+r_up/r_down
    #print (r_up)
    return pred


result = {}
for m in tqdm(ratings['movieId'].drop_duplicates().reset_index(drop=True)):
    result[m] = predict(my_user_id, m)
    #print(result.items())
result = {k: v for k, v in result.items()}